<a href="https://colab.research.google.com/github/Gayathri-Pavuluri/AI-Based-Cyber-Security-Threats-Prediction/blob/main/python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving archive (1).zip to archive (1).zip


In [3]:
import zipfile
import os

# Unzip the uploaded file
with zipfile.ZipFile("archive (1).zip", "r") as zip_ref:
    zip_ref.extractall("cyber_data")

# List extracted files
os.listdir("cyber_data")


['final(2).csv']

In [4]:
# Look at extracted files to identify CSV(s)
for root, dirs, files in os.walk("cyber_data"):
    for file in files:
        if file.endswith(".csv"):
            print(os.path.join(root, file))


cyber_data/final(2).csv


In [6]:
import pandas as pd

df = pd.read_csv("cyber_data/final(2).csv")
df.head()


,Time,Protcol,Flag,Family,Clusters,SeddAddress,ExpAddress,BTC,USD,Netflow_Bytes,IPaddress,Threats,Port,Prediction
0,50,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,500,5,A,Bonet,5061,SS
1,40,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,504,8,A,Bonet,5061,SS
2,30,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,508,7,A,Bonet,5061,SS
3,20,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,512,15,A,Bonet,5061,SS
4,57,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,516,9,A,Bonet,5061,SS


In [7]:
# Check dataset info
df.info()

# Check for missing values
print(df.isnull().sum())

# Check basic statistics
print(df.describe())

# Look at class distribution (if there is a 'label' or 'attack_type' column)
if 'label' in df.columns:
    print(df['label'].value_counts())
elif 'attack_type' in df.columns:
    print(df['attack_type'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149043 entries, 0 to 149042
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Time           149043 non-null  int64 
 1   Protcol        149043 non-null  object
 2   Flag           149043 non-null  object
 3   Family         149043 non-null  object
 4   Clusters       149043 non-null  int64 
 5   SeddAddress    149043 non-null  object
 6   ExpAddress     149043 non-null  object
 7   BTC            149043 non-null  int64 
 8   USD            149043 non-null  int64 
 9   Netflow_Bytes  149043 non-null  int64 
 10  IPaddress      149043 non-null  object
 11  Threats        149043 non-null  object
 12  Port           149043 non-null  int64 
 13  Prediction     149043 non-null  object
dtypes: int64(6), object(8)
memory usage: 15.9+ MB
Time             0
Protcol          0
Flag             0
Family           0
Clusters         0
SeddAddress      0
ExpAddress       0
B

In [8]:
# Fill missing numeric values with median
df = df.fillna(df.median(numeric_only=True))

# Fill categorical missing values with mode
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].fillna(df[col].mode()[0])


In [9]:
from sklearn.preprocessing import LabelEncoder

# Apply label encoding to categorical columns
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [11]:
print(df.columns)


Index(['Time', 'Protcol', 'Flag', 'Family', 'Clusters', 'SeddAddress',
       'ExpAddress', 'BTC', 'USD', 'Netflow_Bytes', 'IPaddress', 'Threats',
       'Port', 'Prediction'],
      dtype='object')


In [12]:
from sklearn.model_selection import train_test_split

# Use 'Prediction' as target column
X = df.drop('Prediction', axis=1)
y = df['Prediction']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop('Prediction', axis=1)
y = df['Prediction']

# Encode categorical columns
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column].astype(str))
        label_encoders[column] = le

# Encode target column if it’s categorical
if y.dtype == 'object':
    le_target = LabelEncoder()
    y = le_target.fit_transform(y.astype(str))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.head()


,Time,Protcol,Flag,Family,Clusters,SeddAddress,ExpAddress,BTC,USD,Netflow_Bytes,IPaddress,Threats,Port
46958,11,2,6,8,1,2,6,11,18454,1246,0,0,5061
137951,6,1,1,8,1,2,4,11,21345,329,2,6,5061
39746,27,2,1,0,1,2,4,11,21345,612,0,8,5062
144462,7,2,7,5,1,3,4,3,735,1427,0,5,5062
79287,7,1,8,7,1,5,6,9,18034,5366,2,0,5066


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Train Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Predictions
y_pred = dt.predict(X_test)

# Evaluation
print("✅ Decision Tree Results")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


✅ Decision Tree Results
Accuracy: 0.994162836727163

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      8403
           1       0.99      0.99      0.99     13373
           2       0.99      0.99      0.99      8033

    accuracy                           0.99     29809
   macro avg       0.99      0.99      0.99     29809
weighted avg       0.99      0.99      0.99     29809


Confusion Matrix:
 [[ 8356    43     4]
 [   45 13291    37]
 [    7    38  7988]]
